In [ ]:
import random
import json
import math
from websocket_server import WebsocketServer
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
class ModelState: 
    def __init__(self, x, y, theta):
        self.x = x
        self.y = y
        self.theta = theta
    
    def __repr__(self):
        return "[x=%.3f, y=%.3f, theta=%.3f]" % (self.x, self.y, self.theta)

class CtrvMotionModel: 
    def __init__(self, stddev):
        self.stddev = stddev
        
    def init(self, x, y, theta):
        x, y, theta = self.gen_noise(x, y, theta)
        return ModelState(x, y, theta)
        
    def predict(self, prev_state, delta_t, vel, yawrate):
        theta = prev_state.theta + yawrate*delta_t
        if math.fabs(yawrate) > 1e-4:
            x = prev_state.x + vel/yawrate * (math.sin(theta) - math.sin(prev_state.theta))
            y = prev_state.y + vel/yawrate * (-math.cos(theta) + math.cos(prev_state.theta))
        else:
            x = prev_state.x + vel * math.cos(prev_state.theta) * delta_t
            y = prev_state.y + vel * math.sin(prev_state.theta) * delta_t
            
        x, y, theta = self.gen_noise(x, y, theta)
        return ModelState(x, y, theta)
    
    def gen_noise(self, x, y, theta):
        return [random.gauss(mean, std) for mean, std in zip([x, y, theta], self.stddev)]


In [ ]:
class Particle:
    def __init__(self, iden, state):
        self.iden = iden
        self.state = state
        self.weight = 0.0
        self.associations = []
        self.sense_x = []
        self.sense_y = []
        
    def predict(self, delta_t, velocity, yawrate, sigma):
        return self


class ParticleFilter:
    n_particles = 10
    
    def __init__(self, motion_model):
        self.particles = []
        self.motion_model = motion_model
        self.is_initialized = False
        
        
    def init(self, sense_x, sense_y, sense_theta):
        for i in range(self.n_particles):
            init_state = self.motion_model.init(sense_x, sense_y, sense_theta)
            self.particles.append(Particle(i, init_state))
            
        self.is_initialized = True
    
    
    def predict(self, delta_t, velocity, yawrate):
        def _predict(particle):
            new_state = motion_model.predict(particle.state, delta_t, velocity, yawrate)
            return Particle(particle.iden, new_state)
        
        self.particles = [_predict(p) for p in self.particles]
        
        
    def find_best(self):
        best_weight = -1.0
        best_part = None
        
        for p in self.particles:
            if p.weight > best_weight:
                best_weight = p.weight
                best_part = p
        
        return p

In [ ]:
class SimulatorResponder:
    valid_prefix = '42'
    sigma_pos = [0.3, 0.3, 0.01]
    delta_t = 0.1
    
    def __init__(self, particle_filter):
        self.pf = particle_filter
    
    def on_connect(self, client, server):
        print("Client connected")
        
    def is_valid_message(self, message):
        return message.startswith(self.valid_prefix)
    
    def parse_data(self, message):
        data_start = len(self.valid_prefix)
        data = json.loads(message[data_start:])
        if data[0] == 'telemetry':
            return data[1]
        else:
            return None
        
    def best_particle(self):
        best = self.pf.find_best()
        return {
          'best_particle_associations': '',
          'best_particle_sense_x': '',
          'best_particle_sense_y': '',
          'best_particle_theta': best.state.theta,
          'best_particle_x': best.state.x,
          'best_particle_y': best.state.y
        }
        
    def respond(self, client, event, data):
        response = self.valid_prefix + json.dumps([event, data])
        client['handler'].send_message(response)
        
    def on_message(self, client, server, message):
        if not self.is_valid_message(message):
            return
        
        data = self.parse_data(message)
        if data is None: 
            self.respond(client, 'manual', {})
            return
        
        if not self.pf.is_initialized:
            sense_x = float(data['sense_x'])
            sense_y = float(data['sense_y'])
            sense_theta = float(data['sense_theta'])
            self.pf.init(sense_x, sense_y, sense_theta)
        else:
            prev_velocity = float(data['previous_velocity'])
            prev_yawrate = float(data['previous_yawrate'])
            self.pf.predict(self.delta_t, prev_velocity, prev_yawrate)

        self.respond(client, 'best_particle', self.best_particle())
        
    

In [ ]:
motion_model = CtrvMotionModel([0.3, 0.3, 0.01])
pf = ParticleFilter(motion_model)
responder = SimulatorResponder(pf)    
server = WebsocketServer(4567, host='127.0.0.1')
server.set_fn_new_client(responder.on_connect)
server.set_fn_message_received(responder.on_message)
server.run_forever()

In [ ]:
sample_data = '42["telemetry",{"sense_x":"15.7544","sense_y":"5.0286","sense_theta":"0.2996","previous_velocity":"5.7666","previous_yawrate":"-0.0145","sense_observations_x":"-5.9299 -1.0073 22.1830 -3.1028 -14.4483 -29.1353 10.3003 -33.2641 -0.2889 44.5006 ","sense_observations_y":"4.3720 -9.4915 4.3068 -25.1741 -22.7379 3.9039 -38.8648 -31.5120 -46.1910 -16.4917 "}]'